# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
import pandas as pd
import numpy as np
from  scipy import stats 

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 
          'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 
          'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 
          'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 
          'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 
          'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 
          'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}



In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import re
    ser = []
    with open('university_towns.txt', 'rt') as con:
        for line in con.readlines(): 
            if(line == '\n'):
                continue
            elif(line.endswith('[edit]\n')):
                k = line.split('[')[0].strip()
                #print(line,'--',k,'>')
            else:
                ret1 = re.search('\(', line)
                ret2 = re.search('\[', line)
                if((ret1 is None)  & (ret2 is None)):
                    val = line.strip()
                    ser.append([k,val])
                elif((ret1 is not None)  & (ret2 is None)):    
                    val = line[0:ret1.span()[0]].strip()
                    ser.append([k,val])
                elif((ret1 is None)  & (ret2 is not None)):
                    val = line[0:ret2.span()[0]].strip()
                    ser.append([k,val]) 
                elif(ret1.span()[0] < ret2.span()[0]):
                    val = line[0:ret1.span()[0]].strip()
                    ser.append([k,val])
                else:
                    val = line[0:ret2.span()[0]].strip()
                    ser.append([k,val])
                #print(line,'--',val,'>')           
                    
                    
    con.close()                
    df = pd.DataFrame(ser, columns = ['State', 'RegionName'])                
    
    return df

df = get_list_of_university_towns()
len(df)
#df.info
df.tail(5)

In [ ]:
def read_qtrly_data():
    df= pd.read_excel('gdplev.xls', skiprows = 219, parse_cols = "E,G", names = ['Qtr', 'GDP'])
    df.index = pd.to_datetime(df['Qtr'])
    df.sort_index(ascending = True, inplace = True)
    return(df)

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = read_qtrly_data()
    for i in range(len(df.index)-2):
        if((df.iloc[i,1] > df.iloc[i + 1,1]) & (df.iloc[i+1,1] > df.iloc[i + 2,1])):
            ans = df.loc[df.index[i], 'Qtr']
            break
        else:
            continue   
    return ans

get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = read_qtrly_data()
    for i in range(len(df.index)-2):
        if(df.index[i] <= pd.to_datetime(get_recession_start())):
            continue
        elif((df.iloc[i,1] < df.iloc[i + 1,1]) & (df.iloc[i+1,1] < df.iloc[i + 2,1])):
            ans = df.loc[df.index[i+2], 'Qtr']
            break
        else:
            continue 
       
    return ans

get_recession_end()

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = read_qtrly_data()
    start = pd.to_datetime(get_recession_start())
    end = pd.to_datetime(get_recession_end())
    subset = df.loc[(df.index >= start) & (df.index <=end), ]
    ans = subset.loc[subset['GDP'] == subset['GDP'].min(),'Qtr'].values[0]
    return ans

get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    homes = pd.read_csv('City_Zhvi_AllHomes.csv') 
    drop_cols = ['RegionID', 'Metro', 'CountyName', 'SizeRank']
    for col in drop_cols:
        homes.drop(col, axis = 'columns', inplace = True)
    homes.set_index('State', inplace = True)
    homes['states'] = pd.Series(states)
    homes.set_index(['states', 'RegionName'], inplace = True)
    homes.columns = pd.to_datetime(homes.columns, format = "%Y-%m")
    #homes_t = homes.transpose().copy()
    homes_sub = homes.loc[:,homes.columns >= pd.to_datetime('2000-01-01')]
    def custom_fun(array_like):
        return array_like[~np.isnan(array_like)].mean()
    grps = homes_sub.transpose().resample('Q').apply(custom_fun)
    grps.index = grps.index.map(lambda x: str(x.year) + 'q' + str(x.quarter))
    result = grps.transpose()
    
    return result

#convert_housing_data_to_quarters()[0:5]
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = get_list_of_university_towns()
    df['UT'] = 'Y'
    df.set_index(['State', 'RegionName'], inplace= True)
    df1 = convert_housing_data_to_quarters()
    df1.index.names = ['State', 'RegionName']
    df2 = pd.merge(df1,df, how = 'left', left_index= True, right_index= True )
    start = get_recession_start()
    bottom = get_recession_bottom()
    ut = df2.loc[df2['UT'] == 'Y',['2008q1','2009q2'] ]
    nut = df2.loc[df2['UT'] != 'Y',['2008q1','2009q2'] ]
    ut_new = ut.dropna(axis= 0,how= 'any') # 257
    nut_new = nut.dropna(axis= 0,how= 'any') # 9599

    ut_new['MPR'] = (ut_new['2008q1'] / ut_new['2009q2']) 
    nut_new['MPR'] = (nut_new['2008q1']/nut_new['2009q2'])
    result = stats.ttest_ind(ut_new['MPR'], nut_new['MPR'])
    p = result.pvalue
    if(p < 0.01):
        different = True
    else:
        different = False
    if(ut_new['MPR'].mean() < nut_new['MPR'].mean()):
        better = "university town"
    else:
        better = "non-university town"
    ans = (different, p, better) 
    
    return ans

run_ttest()